
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gretelai/gretel-blueprints/blob/main/sdk_blueprints/Gretel_Navigator_IAPI_101_Blueprint.ipynb)

<br>

<center><img src="https://gretel-public-website.s3.us-west-2.amazonaws.com/assets/brand/gretel_brand_wordmark.svg" alt="Gretel" width="350"/></center>

<br>

## 👋 Welcome to the Navigator real-time inference API 101 Blueprint!

In this Blueprint, we will introduce the Python SDK for our Navigator real-time inference API, which makes it easy to generate high-quality synthetic tabular and text data with just a few lines of code, powered by [Gretel Navigator](https://gretel.ai/navigator).


<br>

## ✅ Set up your Gretel account

To get started, you will need a [free Gretel account](https://console.gretel.ai/).

<br>

#### Ready? Let's go 🚀

## 💾 Install `gretel-client` and its dependencies

In [ ]:
%%capture
!pip install gretel-client

## 🛜 Configure your Gretel session

- [The Gretel object](https://docs.gretel.ai/create-synthetic-data/gretel-sdk/the-gretel-object) provides a high-level interface for streamlining interactions with Gretel's APIs.

- Retrieve your Gretel API key [here](https://console.gretel.ai/users/me/key).

In [ ]:
from gretel_client import Gretel

gretel = Gretel(api_key="prompt", validate=True)

## 🚀 Real-time inference API

- The Navigator real-time inference API makes it possible to programmatically run Navigator outside the [Gretel Console](https://console.gretel.ai/navigator).

- Our [Python SDK](https://github.com/gretelai/gretel-python-client) provides an intuitive high-level interface for the Navigator API.

- Navigator currently supports two data generation modes: `"tabular"` and `"natural_language"`. In both modes, you can choose the backend model that powers the generation.

In [ ]:
# list "tabular" backend models
gretel.factories.get_navigator_model_list("tabular")

In [ ]:
# list "natural_language" backend models
gretel.factories.get_navigator_model_list("natural_language")

**Notes:**

- `gretelai/auto` automatically selects the current default model, which will change with time as models continue to evolve.

- The `factories` attribute of the `Gretel` object provides methods for creating new objects that interact with Gretel's APIs.

## 📊 Tabular data generation

- We use the `initialize_navigator_api` method of the `factories` attribute to create a Navigator API object.

- With `model_type = "tabular"` (which is the default), we initialize Navigator's tabular API.

- To select a different backend model, use the optional `backend_model` argument, which defaults to `gretelai/auto`.

In [ ]:
tabular = gretel.factories.initialize_navigator_api("tabular")

In [ ]:
# generate tabular data from a natural language prompt

prompt = """\
Generate customer bank transaction data. Include the following columns:
- customer_name
- customer_id
- transaction_date
- transaction_amount
- transaction_type
- transaction_category
- account_balance
"""

df = tabular.generate(prompt, num_records=25)

df

In [ ]:
# add column to the generated table using the `edit` method

prompt = """\
Add the following column to the provided table:

- customer_address
"""

df_edited = tabular.edit(prompt, seed_data=df)

df_edited

## 📚 Natural language data generation

- With `model_type = "natural_language"`, we initialize Navigator's natural language API.

- To select a different backend model, use the optional backend_model argument, which defaults to `gretelai/gpt-auto`.

In [ ]:
llm = gretel.factories.initialize_navigator_api("natural_language")

In [ ]:
# generate text from a natural language prompt
llm.generate("Please tell me a funny joke about data scientists.")

In [ ]:
# let's see if the LLM is funnier with a higher generation temperature
llm.generate("Please tell me a funny joke about data scientists.", temperature=2)

## 🤖 Combining Navigator's APIs

- Navigator's APIs can be combined to dynamically create/augment prompts for tabular data generation.  

In [ ]:
# use the "natural_language" API to generate the fields of a tabular dataset

dataset_type = "insurance fraud"


llm_prompt = f"""\
List 8 fields you expect to find in an {dataset_type} dataset.

Important: Include descriptions for each field. Please do not explain yourself.
"""

fields = llm.generate(llm_prompt, temperature=0.1)

print(fields)

In [ ]:
# use the "tabular" API to generate a dataset using the above generated fields

final_prompt = f"""\
Create an {dataset_type} dataset with the following fields:

{fields}
"""

print(final_prompt, end="\n\n")

df = tabular.generate(final_prompt, num_records=25)

df